<a href="https://colab.research.google.com/github/jackyluo-aiit/colab_files/blob/master/TF_IDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive
!ls
import os
os.chdir('drive/Dataset')
!ls

adc.json  drive  sample_data
doc2vec_model
drive
epoch10_mincount500_window5_dm_alpha025_worker7_minalpha001_model_v2
epoch10_mincount500_window5_dm_alpha025_worker7_minalpha001_model_v2.docvecs.vectors_docs.npy
LSH_data.txt
preprocessed_data.csv
sentiment_cleaned.csv
stream_data.txt
testSet2.txt
testSet.txt
training.1600000.processed.noemoticon.csv


In [0]:
!pip install tweet-preprocessor

  Created wheel for tweet-preprocessor: filename=tweet_preprocessor-0.5.0-cp36-none-any.whl size=7947 sha256=6b8593506dfe13464019623094bde4eb53ff5e7d78519a2493f6cf8e0a16d882
  Stored in directory: /root/.cache/pip/wheels/1b/27/cc/49938e98a2470802ebdefae9d2b3f524768e970c1ebbe2dc4a
Successfully built tweet-preprocessor


In [0]:
import re
import numpy as np
import pandas as pd
import preprocessor as p
from nltk.stem.porter import *
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import csv

'''
This file is to select the data with polarity are '0' and '4';
Then clean the data, which only remains the words and punctuations.
'''
# data = pd.read_csv(
#     "training.1600000.processed.noemoticon.csv",
#     encoding="ISO-8859-1")

# data.columns = ['label', '', '', '', '', 'content']
# sel_data = data[(data['label'] == 0) | (data['label'] == 4)]
# print("selected 0 and 4 data:", sel_data)
# sel_content = sel_data['content']
# content = []

# for line in range(len(sel_content)):
#     clean_line = p.clean(sel_content[line])
#     # print(clean_line)
#     content.append(clean_line)

# # tfidf_vectorizer = TfidfVectorizer()
# # x = tfidf_vectorizer.fit_transform(content)
# # print(tfidf_vectorizer.get_feature_names())
# # # print(x.toarray().shape())
# #
# # # print(x.shape())
# # # print(data[['label', 'content']])
# # vector = pd.DataFrame(x.toarray())
# # print(vector)


# new_data = sel_data[['label']]
# new_data.insert(1, 'content', content)
# print("new data:", new_data.head)



# def remove_pattern(input_txt, pattern):
#     r = re.findall(pattern, input_txt)
#     for i in r:
#         input_txt = re.sub(i, '', input_txt)

#     return input_txt


# new_data['clean_data'] = np.vectorize(remove_pattern)(new_data['content'], "@[\w]*")
# new_data['clean_data'] = new_data['clean_data'].str.replace("[^a-zA-Z#]", " ")
# new_data['clean_data'] = new_data['clean_data'].apply(lambda x: ' '.join([w for w in x.split() if len(w) > 3]))
# tokenized_content = new_data['clean_data'].apply(lambda x: x.split())

# stemmer = PorterStemmer()
# tokenized_content = tokenized_content.apply(lambda x: [stemmer.stem(i) for i in x])
# for i in range(len(tokenized_content)):
#     tokenized_content[i] = ' '.join(tokenized_content[i])

# new_data['clean_data'] = tokenized_content
# print("after tokenized:", new_data)
# print("new data len:", len(new_data))
# unique_content = new_data.content.unique()
# print("content len:", new_data.content.count())
# print("unique content len:", len(unique_content))
# unique_clean_data = new_data.clean_data.unique()
# print("unique clean content len:", len(unique_clean_data))

# print("Empty content:", new_data[new_data.content.isnull()])
# print("Empty clean_content:", new_data[new_data.clean_data.isnull()])

# new_data.drop_duplicates(subset=['clean_data'], keep='first', inplace=True)
# new_data.reset_index(drop=True, inplace=True)
# new_data['clean_data_length'] = new_data['clean_data'].apply(len)
# print(new_data.info())
# print("new data:\n", new_data.loc[:,('label','content','clean_data')])
# new_data.to_csv("preprocessed_data.csv")
new_data = pd.read_csv("preprocessed_data.csv")
print(new_data.loc[:, ('clean_data')])

tf_idf_vect = CountVectorizer(analyzer='word',ngram_range=(1,1),stop_words='english', min_df = 0.0001)
tf_idf_vect.fit(new_data['clean_data'].values.astype('U'))
desc_matrix = tf_idf_vect.transform(new_data["clean_data"].values.astype('U'))

num_clusters = 2
km = KMeans(n_clusters=num_clusters)
km.fit(desc_matrix)
clusters = km.labels_.tolist()

tweets = {'Tweet': new_data["clean_data"].tolist(), 'cluster': clusters}
cluster_frame = pd.DataFrame(tweets, index = [clusters])
print("after clustering:\n",cluster_frame)

label_dict = {i: np.where(km.labels_ == i)[0] for i in range(km.n_clusters)}
print("cluster label set:\n", label_dict)
cluster_fp_set = {}
for key in label_dict.keys():
  count = 0
  cluster_set = label_dict[key]
  for item in cluster_set:
    if key != 0:
      key = 4
    if new_data.loc[item, ['label']].values != key:
      count+=1
    cluster_fp = count/len(cluster_set)
    cluster_fp_set[key] = cluster_fp

# result_frame = pd.DataFrame(cluster_fp_set, )

print("fpp:\n",cluster_fp_set)
with open('result_TFIDF.csv','wb') as f:
    w = csv.writer(f)
    w.writerows(cluster_fp_set.items())


















0          upset that updat facebook text might result sc...
1                  dive mani time ball manag save rest bound
2                            whole bodi feel itchi like fire
3                               behav here becaus over there
4                                                 whole crew
                                 ...                        
1435443    yeah that doe work better than just wait just ...
1435444                 just woke have school best feel ever
1435445                 thewdb veri cool hear walt interview
1435446                        readi your mojo makeov detail
1435447          happi birthday alll time tupac amaru shakur
Name: clean_data, Length: 1435448, dtype: object
after clustering:
                                                 Tweet  cluster
1   upset that updat facebook text might result sc...        1
1           dive mani time ball manag save rest bound        1
1                     whole bodi feel itchi like fire        1
1        

NameError: ignored